Les films les mieux notés et les caractéristiques qu’ils partagent.

In [49]:
# IMPORTS
import pandas as pd

df_title_ratings = pd.read_csv("./BD/title.ratings.tsv.gz", compression = 'gzip', sep = '\t')
df_tmdb_full = pd.read_csv("./BD/tmdb_full.csv")

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_1684/364869041.py:5: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb_full = pd.read_csv("./BD/tmdb_full.csv")


In [89]:
# MERGES
 
df_ratings = pd.merge(df_title_ratings,
                      df_tmdb_full,
                      how = 'outer',
                      left_on = 'tconst',
                      right_on = 'imdb_id') 

In [90]:
# Clean du Merge

df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [68]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]


In [69]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [70]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_1684/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [74]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [76]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['vote_exact'] = df_a_travailler_3[['vote_average', 'averageRating']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_1684/4251311695.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = df_a_travailler_3[['vote_average', 'averageRating']].max(axis = 1)


In [77]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_1684/2539145554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['vote_average', 'averageRating']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_1684/2539145554.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['vote_average', 'averageRating']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_1684/25391455

In [78]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])